In [19]:
import xml.etree.ElementTree as ET
from dataclasses import dataclass
from typing import List
import pandas as pd

In [20]:
@dataclass
class AircraftInfo:
    """Simplified class to store essential aircraft information"""
    callsign: str
    aircraft_type: str
    start_time: int
    air_routes: List[str]
    initial_latitude: str
    initial_longitude: str
    initial_altitude: float  # In flight level
    requested_flight_level: float

def parse_essential_aircraft_data(xml_content: str) -> List[AircraftInfo]:
    """
    Parse only essential aircraft information from XML content
    
    Args:
        xml_content (str): XML content as string
        
    Returns:
        List[AircraftInfo]: List of AircraftInfo objects with essential data
    """
    root = ET.fromstring(xml_content)
    aircraft_list = []
    
    for flightplan in root.findall('initial-flightplans'):
        # Extract basic information
        callsign = flightplan.find('callsign').text
        aircraft_type = flightplan.find('type').text
        start_time = int(flightplan.find('time').text)
        
        # Extract air routes
        air_routes = [route.text for route in flightplan.findall('air_route')]
            
        # Extract initial position information
        init_element = flightplan.find('init')
        init_lat = init_element.find('pos/lat').text
        init_lon = init_element.find('pos/lon').text
        init_alt = float(init_element.find('alt').text)  # In FL
        
        # Get requested flight level
        rfl = float(flightplan.find('rfl').text)
        
        # Create AircraftInfo object
        aircraft = AircraftInfo(
            callsign=callsign,
            aircraft_type=aircraft_type,
            start_time = start_time,
            air_routes=air_routes,
            initial_latitude=init_lat,
            initial_longitude=init_lon,
            initial_altitude=init_alt,
            requested_flight_level=rfl
        )
        
        aircraft_list.append(aircraft)
    
    return aircraft_list

def convert_to_dataframe(aircraft_list: List[AircraftInfo]) -> pd.DataFrame:
    """
    Convert list of AircraftInfo objects to a pandas DataFrame
    """
    data = []
    for aircraft in aircraft_list:
        aircraft_dict = {
            'Callsign': aircraft.callsign,
            'Start Time': aircraft.start_time,
            'Aircraft Type': aircraft.aircraft_type,
            'Air Routes': ', '.join(aircraft.air_routes),
            'Initial Latitude': aircraft.initial_latitude,
            'Initial Longitude': aircraft.initial_longitude,
            'Initial Altitude (FL)': aircraft.initial_altitude,
            'Requested FL': aircraft.requested_flight_level
        }
        data.append(aircraft_dict)
    
    return pd.DataFrame(data)



In [21]:

with open('complex_scenario_24jan.xdat', 'r') as file:
    xml_content = file.read()
    
# Parse the data
aircraft_list = parse_essential_aircraft_data(xml_content)

# Convert to DataFrame
df = convert_to_dataframe(aircraft_list)

# Display results
print("\nAs DataFrame:")
print(df)

print("\nAs objects:")
for aircraft in aircraft_list[:2]:  # Show first two aircraft
    print(f"\nAircraft: {aircraft.callsign}")
    print(f"Type: {aircraft.aircraft_type}")
    print(f"Routes: {' -> '.join(aircraft.air_routes)}")
    print(f"Position: {aircraft.initial_latitude}, {aircraft.initial_longitude}")
    print(f"Initial FL: {aircraft.initial_altitude}")
    print(f"Requested FL: {aircraft.requested_flight_level}")
print("\nAs objects:")
for aircraft in aircraft_list[:2]:  # Show first two aircraft
    print(f"\nAircraft: {aircraft.callsign}")
    print(f"Type: {aircraft.aircraft_type}")
    print(f"Routes: {' -> '.join(aircraft.air_routes)}")
    print(f"Position: {aircraft.initial_latitude}, {aircraft.initial_longitude}")
    print(f"Initial FL: {aircraft.initial_altitude}")
    print(f"Requested FL: {aircraft.requested_flight_level}")


As DataFrame:
   Callsign  Start Time Aircraft Type  \
0      SQ10           0          B738   
1      SQ11         150          B738   
2      SQ12         300          B738   
3      SQ13         100          A320   
4      SQ14         350          A320   
5      SQ15         600          A320   
6      SQ16           0          B738   
7      SQ17         150          B738   
8      SQ18         350          B738   
9      SQ19         550          B738   
10    SQ110         100          A320   
11    SQ111         250          A320   
12    SQ112         400          A320   
13    SQ113         150          B738   
14    SQ114         300          B738   
15    SQ115         450          B738   
16    SQ116           0          B734   
17    SQ117         150          B734   
18    SQ118         300          B734   
19    SQ119         450          B734   
20    SQ120         600          B734   

                                           Air Routes Initial Latitude  \
0       

In [22]:
df.to_csv('aircraft_data_24jan.csv')